In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.preprocessing import StandardScaler
import pandas as pd


In [31]:
lab = pd.read_csv("lab_sBert_concat_fulldata.csv")
print(lab.columns)
#hours = pd.read_csv("lab_hours.csv")
#lab["hour"] = hours["hour"]
targets = lab[["m:icustayid", "sepsis_label", "hour", "o:age", "o:gender"]]
rows = lab.drop(columns=["clean_text","Unnamed: 0.2","Unnamed: 0.1","Unnamed: 0", "o:age", "o:gender","m:icustayid", "sepsis_label", "hour","m:charttime", "traj", "step", "r:reward", "a:action","Unnamed: 0", "Unnamed: 0.1", "m:presumed_onset", "o:cumulated_balance", "o:re_admission", "o:output_4hourly", "o:output_total", "o:PaO2_FiO2", "o:input_total"])
scaler = StandardScaler()
rows[rows.columns] = scaler.fit_transform(rows[rows.columns])
rows["m:icustayid"] = targets["m:icustayid"]
rows["sepsis_label"] = targets["sepsis_label"]
rows["hour"] = targets["hour"]
rows["o:age"] = targets["o:age"]
rows["o:gender"] = targets["o:gender"]


min_max = pd.DataFrame({"min": rows.min(), "max": rows.max()})
min_max

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'a:action', 'm:charttime',
       'm:icustayid', 'm:presumed_onset', 'o:Arterial_BE',
       'o:Arterial_lactate', 'o:Arterial_pH', 'o:BUN', 'o:Calcium',
       'o:Chloride', 'o:Creatinine', 'o:DiaBP', 'o:FiO2_1', 'o:GCS',
       'o:Glucose', 'o:HCO3', 'o:HR', 'o:Hb', 'o:INR', 'o:Magnesium',
       'o:MeanBP', 'o:PT', 'o:PTT', 'o:PaO2_FiO2', 'o:Platelets_count',
       'o:Potassium', 'o:RR', 'o:SGOT', 'o:SGPT', 'o:SIRS', 'o:SOFA',
       'o:Shock_Index', 'o:Sodium', 'o:SpO2', 'o:SysBP', 'o:Temp_C',
       'o:Total_bili', 'o:WBC_count', 'o:Weight_kg', 'o:age',
       'o:cumulated_balance', 'o:gender', 'o:input_4hourly', 'o:input_total',
       'o:max_dose_vaso', 'o:mechvent', 'o:output_4hourly', 'o:output_total',
       'o:paCO2', 'o:paO2', 'o:re_admission', 'r:reward', 'step', 'traj',
       'sepsis_label', 'hour', 'clean_text', 'sBert:0', 'sBert:1', 'sBert:2',
       'sBert:3', 'sBert:4', 'sBert:5', 'sBert:6', 'sBert:7', 'sBert:8',


,min,max
o:Arterial_BE,-10.902151,25.890943
o:Arterial_lactate,-1.551057,20.392145
o:Arterial_pH,-72.425298,6.610118
o:BUN,-7.955757,4.094832
o:Calcium,-11.464160,15.463885
...,...,...
m:icustayid,200003.000000,299999.000000
sepsis_label,0.000000,1.000000
hour,0.000000,78.000000
o:age,-2.775468,1.616612


In [32]:
for i in rows.columns:
    print(i)

o:Arterial_BE
o:Arterial_lactate
o:Arterial_pH
o:BUN
o:Calcium
o:Chloride
o:Creatinine
o:DiaBP
o:FiO2_1
o:GCS
o:Glucose
o:HCO3
o:HR
o:Hb
o:INR
o:Magnesium
o:MeanBP
o:PT
o:PTT
o:Platelets_count
o:Potassium
o:RR
o:SGOT
o:SGPT
o:SIRS
o:SOFA
o:Shock_Index
o:Sodium
o:SpO2
o:SysBP
o:Temp_C
o:Total_bili
o:WBC_count
o:Weight_kg
o:input_4hourly
o:max_dose_vaso
o:mechvent
o:paCO2
o:paO2
sBert:0
sBert:1
sBert:2
sBert:3
sBert:4
sBert:5
sBert:6
sBert:7
sBert:8
sBert:9
sBert:10
sBert:11
sBert:12
sBert:13
sBert:14
sBert:15
sBert:16
sBert:17
sBert:18
sBert:19
sBert:20
sBert:21
sBert:22
sBert:23
sBert:24
sBert:25
sBert:26
sBert:27
sBert:28
sBert:29
sBert:30
sBert:31
m:icustayid
sepsis_label
hour
o:age
o:gender


In [33]:
strats_data = (rows.melt(id_vars=['hour', 'm:icustayid', 'sepsis_label'], var_name='variable',value_name='value', ignore_index=False)
       .sort_values(['m:icustayid', 'hour'])
       .reset_index(drop=True))
mean_stds = strats_data.groupby('variable').agg({'value':['mean', 'std']})
mean_stds.columns = [col[1] for col in mean_stds.columns]
mean_stds

,mean,std
variable,,
o:Arterial_BE,4.469890e-18,1.0
o:Arterial_lactate,-2.599263e-17,1.0
o:Arterial_pH,2.502550e-17,1.0
o:BUN,2.199586e-17,1.0
o:Calcium,-8.757626e-18,1.0
...,...,...
sBert:5,6.894529e-17,1.0
sBert:6,1.767813e-16,1.0
sBert:7,7.081313e-16,1.0


In [34]:
strats_data = strats_data.merge(mean_stds.reset_index(), on='variable', how='left')
strats_data['ts_ind'] = strats_data.groupby(['m:icustayid']).ngroup()
strats_data

,hour,m:icustayid,sepsis_label,variable,value,mean,std,ts_ind
0,0.0,200003,1,o:Arterial_BE,1.071196,4.469890e-18,1.000000,0
1,0.0,200003,1,o:Arterial_lactate,-0.560318,-2.599263e-17,1.000000,0
2,0.0,200003,1,o:Arterial_pH,1.078780,2.502550e-17,1.000000,0
3,0.0,200003,1,o:BUN,-0.381145,2.199586e-17,1.000000,0
4,0.0,200003,1,o:Calcium,0.095615,-8.757626e-18,1.000000,0
...,...,...,...,...,...,...,...,...
144405529,38.0,299999,0,sBert:29,0.391223,4.413519e-17,1.000000,41773
144405530,38.0,299999,0,sBert:30,-0.636355,5.293132e-16,1.000000,41773
144405531,38.0,299999,0,sBert:31,-0.439726,-2.048425e-16,1.000000,41773
144405532,38.0,299999,0,o:age,-0.849940,4.296106e-04,0.999406,41773


In [35]:
# build oc
oc = targets.groupby('m:icustayid').head(1).reset_index(drop=True)
oc['ts_ind'] = oc.groupby(['m:icustayid']).ngroup()
oc["SUBJECT_ID"] = oc["m:icustayid"]
#targets_oc = targets.loc[targets.SUBJECT_ID.isin(train_val_IDs)]
oc


,m:icustayid,sepsis_label,hour,o:age,o:gender,ts_ind,SUBJECT_ID
0,202134,1,0.0,-0.494919,-0.5,889,202134
1,293407,1,0.0,0.462969,-0.5,39010,293407
2,222148,0,0.0,0.469747,0.5,9303,222148
3,293525,1,0.0,-1.866752,0.5,39051,293525
4,246035,0,0.0,-0.879073,0.5,19215,246035
...,...,...,...,...,...,...,...
41769,280210,1,0.0,0.001012,-0.5,33538,280210
41770,216102,0,0.0,-0.151317,0.5,6788,216102
41771,252411,0,0.0,-0.746649,0.5,21926,252411
41772,202836,0,0.0,0.851900,-0.5,1192,202836


In [36]:
ids = oc['SUBJECT_ID'].tolist()
labels = oc['sepsis_label'].tolist()

new_patient_ids = []
new_labels = []

for i in range(len(labels)):
  # print(i)
  if ids[i] in new_patient_ids:
    continue
  else:
    new_patient_ids.append(ids[i])
    new_labels.append(labels[i])

In [37]:
from collections import Counter
# data ratio
Counter(labels)

Counter({0: 23801, 1: 17973})

In [38]:
from sklearn.model_selection import train_test_split

x, x_test, y, y_test = train_test_split(new_patient_ids, new_labels, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)
# train
train_ind = []

ts_ind = oc['ts_ind'].tolist()
# ids = ids

for i in range(len(ts_ind)):
  if ids[i] in x_train:
    train_ind.append(ts_ind[i])

# number of train patients
print(len(x_train))
# number of train instances
print(len(train_ind))
# to np.array
train_ind = np.array(train_ind)

test_ind = []

for i in range(len(ts_ind)):
  if ids[i] in x_test:
    test_ind.append(ts_ind[i])

# number of test patients
print(len(x_test))
# number of test instances
print(len(test_ind))
# to np.array
test_ind = np.array(test_ind)

valid_ind = []

for i in range(len(ts_ind)):
  if ids[i] in x_val:
    valid_ind.append(ts_ind[i])

# number of test patients
print(len(x_val))
# number of test instances
print(len(valid_ind))
# to np.array
valid_ind = np.array(valid_ind)

strats_data = strats_data.drop(columns="sepsis_label")
strats_data = strats_data.dropna()

data = strats_data[["ts_ind", "hour","variable", "value", "mean", "std"]]
data

26735
26735
8355
8355
6684
6684


,ts_ind,hour,variable,value,mean,std
0,0,0.0,o:Arterial_BE,1.071196,4.469890e-18,1.000000
1,0,0.0,o:Arterial_lactate,-0.560318,-2.599263e-17,1.000000
2,0,0.0,o:Arterial_pH,1.078780,2.502550e-17,1.000000
3,0,0.0,o:BUN,-0.381145,2.199586e-17,1.000000
4,0,0.0,o:Calcium,0.095615,-8.757626e-18,1.000000
...,...,...,...,...,...,...
144405529,41773,38.0,sBert:29,0.391223,4.413519e-17,1.000000
144405530,41773,38.0,sBert:30,-0.636355,5.293132e-16,1.000000
144405531,41773,38.0,sBert:31,-0.439726,-2.048425e-16,1.000000
144405532,41773,38.0,o:age,-0.849940,4.296106e-04,0.999406


In [39]:
pred_window = 1 # hours that the output vector represents. 1 because i want to learn to predict 1 hour many times
obs_windows = [16]

In [40]:
# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID'], inplace=True)
# Fix age.
#data.loc[(data.variable=='o:age')&(data.value>200), 'value'] = 91.4
# Get static data with mean fill and missingness indicator.
static_varis = ['o:age', 'o:gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
#means = demo.mean(axis=0, keepdims=True)
#stds = demo.std(axis=0, keepdims=True)
#stds = (stds==0)*1 + (stds!=0)*stds
#demo = (demo-means)/stds
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

3160776it [00:01, 1709811.20it/s]
100%|██████████| 1/1 [00:18<00:00, 18.24s/it]


In [41]:
data = strats_data[["ts_ind", "hour","variable", "value", "mean", "std"]]
oc = targets.groupby('m:icustayid').head(1).reset_index(drop=True)
oc['ts_ind'] = oc.groupby(['m:icustayid']).ngroup()
oc["SUBJECT_ID"] = oc["m:icustayid"]
#targets_oc = targets.loc[targets.SUBJECT_ID.isin(train_val_IDs)]
data = data.dropna()
oc = oc.dropna()

# train
train_ind = []
ts_ind = oc['ts_ind'].tolist()
# ids = ids
for i in range(len(ts_ind)):
  if ids[i] in x_train:
    train_ind.append(ts_ind[i])
# number of train patients
print(len(x_train))
# number of train instances
print(len(train_ind))
# to np.array
train_ind = np.array(train_ind)
test_ind = []
for i in range(len(ts_ind)):
  if ids[i] in x_test:
    test_ind.append(ts_ind[i])
# number of test patients
print(len(x_test))
# number of test instances
print(len(test_ind))
# to np.array
test_ind = np.array(test_ind)
valid_ind = []
for i in range(len(ts_ind)):
  if ids[i] in x_val:
    valid_ind.append(ts_ind[i])
# number of test patients
print(len(x_val))
# number of test instances
print(len(valid_ind))
# to np.array
valid_ind = np.array(valid_ind)


26735
26735
8355
8355
6684
6684


In [42]:
# # Read data.
# data_path = './../mimic_iii_preprocessed.pkl'
# data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour>=0)&(data.hour<=24)]
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Fix age.
#data.loc[(data.variable=='o:age')&(data.value>200), 'value'] = 91.4
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['sepsis_label']).astype('float32')
N = data.ts_ind.max() + 1
# Get static data with mean fill and missingness indicator.
static_varis = ['o:age', 'o:gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
#means = demo.mean(axis=0, keepdims=True)
#stds = demo.std(axis=0, keepdims=True)
#stds = (stds==0)*1 + (stds!=0)*stds
#demo = (demo-means)/stds
# Trim to max len.
#data = data.sample(frac=1)
data = data.groupby('ts_ind').head(880)
# Get N, V, var_to_ind.
N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
V = len(varis)
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index':'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print ('max_len', max_len)
# Generate times_ip and values_ip matrices.
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    times_inp[ts_ind, l] = row.hour
    values_inp[ts_ind, l] = row.value
    varis_inp[ts_ind, l] = row.vind
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
# Generate 3 sets of inputs and outputs.
train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
del times_inp, values_inp, varis_inp
train_op = y[train_ind]
valid_op = y[valid_ind]
test_op = y[test_ind]
del y

1707398it [00:00, 1732178.67it/s]


max_len 880


35635143it [00:27, 1298101.63it/s]


In [43]:
mape = tf.keras.losses.MeanAbsolutePercentageError()
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

########################################################################################################
########################################################################################################
class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
def mortality_loss(y_true, y_pred):
    sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.mean(sample_weights*bce, axis=-1)
########################################################################################################
########################################################################################################

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def mape_fore(y_true, y_pred):
    truth = y_true[:,V:]
    pred = y_pred
    return mape(truth, pred)

                                          
def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [44]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn
import smart_cond_mod as sc


class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()

    def build(self, input_shape):
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)

    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)


class Attention(Layer):

    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)


class Transformer(Layer):

    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        #self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
         #                        initializer='glorot_uniform', trainable=True)
        #self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
         #                        initializer='glorot_uniform', trainable=True)
        #self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
         #                        initializer='glorot_uniform', trainable=True)
        #self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
         #                        initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)

    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            """for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                print(A.shape) # 880,880"""
            x_complex = tf.cast(x, tf.complex64)
            fft_hidden = tf.signal.fft(x_complex)
            fft_seq = tf.math.real(tf.signal.fft(fft_hidden))
            #print(fft_seq.shape)
                #mha_ops.append(fft_seq) #jeder teil von mha_ops hat shape 880,12 -> 880,48
            #print(len(mha_ops))
            #conc = K.concatenate(mha_ops, axis=-1)
            #conc (None, 880, 48)
            #proj = K.dot(conc, self.Wo[i,:,:])
            #proj (None, 880, 50)
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(fft_seq, rate=self.dropout)),\
                                       lambda: array_ops.identity(fft_seq))
            # Add & LN
            #print(x.shape, proj.shape) #orig x 880,50 proj 880,50
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])),
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]
        return x

    """def call(self, x, mask, mask_value=-1e-30):
        print(x.shape)
        tf.math.real(tf.signal.fft2d(x))
        fft_hidden = torch.fft.fft(x, dim=2)
        fft_seq = torch.fft.fft(fft_hidden, dim=0)
        return torch.real(fft_seq)"""
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    #cont_emb_complex = tf.cast(cont_emb, tf.complex64)
    #fourier = tf.math.real(tf.signal.fft2d(cont_emb_complex)) 
    #fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, fourier])
    conc = Concatenate(axis=-1)([fused_emb, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis], op)
    if forecast:
        fore_model = Model([demo, times, values, varis], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

In [45]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
lr, batch_size, samples_per_epoch, patience = 0.0005, 32, len(fore_train_op), 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
print(fore_model.summary())
# fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
lossfunction = mape_fore #forecast_loss
fore_model.compile(loss=lossfunction, optimizer=Adam(lr))

# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
fore_savepath = 'STraTSBERT_microsoft_data_Fourier.h5'
train_losses = []
val_losses = []
for e in range(1000):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # pre-train data
        e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
        pbar.set_description('%f'%(e_loss/(start+1)))
    val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
    train_losses.append(e_loss*batch_size/samples_per_epoch)
    val_losses.append(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        fore_model.save_weights(fore_savepath)
        best_epoch = e
    if (e-best_epoch)>patience:
        losses = pd.DataFrame({'train_loss': train_losses,'val_loss': val_losses})
        losses.to_csv(f"losses_{fore_savepath}.csv")
        break

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 880)]                0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 880)]                0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 880)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 880, 50)              3600      ['input_6[0][0]']             
                                                                                            

None


  0%|          | 0/939 [00:00<?, ?it/s]

2.121915: 100%|█████████▉| 938/939 [18:14<00:01,  1.16s/it]  

2.121894: 100%|██████████| 939/939 [18:16<00:00,  1.17s/it]


353/353 [==============================] - 200s 565ms/step - loss: 64.9375
Epoch 0 loss 67.83506395337045 val loss 64.93750762939453


2.071180:   5%|▍         | 43/939 [00:51<17:52,  1.20s/it]


KeyboardInterrupt: 

: 

maybe bad performance expected cause bert embeddings not representable feature cause meant for downstream task, however sentence bert embeddings should represent text e.g. similar text similar embedding vector space, maybe thats why tfidf better. because it actually represents words e.g. probabilities of septic texts etc.